# HW5 due 5p Fri Feb 12 2020

This assignment will be graded on both participation and correctness (1 point each, for a total of 2 points for each sub-problem).

You are welcome (and encouraged) to:
* work together, synchronously and asynchronously, in study groups;
* use analytical and numerical computational tools -- specify the tool(s) in sourcecode and/or text;
* reuse example sourcecode and other materials provided in this course;
* consult textbooks, websites, and other publicly-available materials -- include full citation(s) with the URL and/or DOI.

Submit your homework writeup by uploading a .pdf and/or .ipynb on the Canvas Assignment.

You are welcome (and encouraged) to typeset your homework assignments rather than write them by hand.  While you could do this with LaTeX, you may find it easier to use the Colaboratory Notebook, since it is adept at embedding equations (via LaTeX syntax), matrix computations, and control system calculations.



# 0. [preferred name]; [preferred pronouns]

a. Approximately how many hours did you spend on this assignment?

***Solution:*** $\approx 1.5$ hours

b. Were there specific problems that took much longer than others?

***Solution:*** Problem 2 -- mostly because it had more subproblems.

c. What class meeting(s) did you participate in this week?

***Solution:*** all :)

d. What timezone(s) were you working in this week?

***Solution:*** Seattle time

In [1]:
# numpy = numerical Python, implements arrays (/ matrices)
import numpy as np
# limit number of decimal places printed for floating-point numbers
np.set_printoptions(precision=3)

# scipy = scientific Python, implements operations on arrays / matrices
import scipy as sp
# linalg = linear algebra, implements eigenvalues, matrix inverse, etc
from scipy import linalg as la
# optimize = optimization, root finding, etc
from scipy import optimize as op

# produce matlab-style plots
import matplotlib as mpl
# increase font size on plots
mpl.rc('font',**{'size':18})
# use LaTeX to render symbols
mpl.rc('text',usetex=False)
# animation
from matplotlib import animation as ani
# Matlab-style plotting
import matplotlib.pyplot as plt

# symbolic computation, i.e. computer algebra (like Mathematica, Wolfram Alpha)
import sympy as sym

# os = operating system; access OS-level commands
# e.g. create directory, delete file, execute command
# (more platform-independent than "!")
import os

# test whether this is a Colaboratory or Jupyter notebook
try:
  import google.colab
  COLAB = True
  print('Colaboratory Notebook')
except:
  COLAB = False
  print('Jupyter Notebook')

# Colab notebook
if COLAB:  
  # render SymPy equations nicely in Colaboratory Notebook
  def colab_latex_printer(exp,**options):
    from google.colab.output._publish import javascript
    url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/latest.js?config=default"
    javascript(url=url)
    return sym.printing.latex(exp,**options)
  
  sym.init_printing(use_latex="mathjax",latex_printer=colab_latex_printer)

# Jupyter notebook
else:
  sym.init_printing(use_latex='mathjax')

# SciPy module that implements many of the routines in ctrl
from scipy import signal as sig

Colaboratory Notebook


# 1. satellite control

Linearizing the dynamics of a satellite along an equilibrium yields

$$ \dot{x} = A x + B u = \left[ \begin{array}{cccc} 0 & 1 & 0 & 0 \\ 3\omega^2 & 0 & 0 & 2\omega \\ 0 & 0 & 0 & 1 \\ 0 & -2\omega & 0 & 1 \end{array} \right] x + \left[ \begin{array}{cccc} 0 & 0 \\ 1 & 0 \\ 0 & 0 \\ 0 & 1 \end{array} \right] u; $$

the state vector $x\in\mathbb{R}^4$ consists of radius $x_1$, radial velocity $x_2$, angle $x_3$, and angular velocity $x_4$;
the input vector $u\in\mathbb{R}^2$ consists of a radial thruster $u_1$ and a tangential thruster $u_2$.

(a.) Show that the system is controllable.

***Solution:*** we will use the controllability matrix 

$$ \mathcal{C} = \left[\begin{array}{cccc} B & AB & A^2 B & A^3 B \end{array}\right] $$

to assess controllability -- the system is controllable if (and only if) $\operatorname{rank}\mathcal{C} = 4$.  Since rank equals column rank, which equals the number of linearly independent columns, it suffices to work from left-to-right in the controllability matrix, assessing linear independence of columns.  In particular, the two columns of $B$ are linearly independent, so we need to find two more linearly independent columns among the six columns of $AB$, $A^2B$, $A^3B$.

Let's start with $AB$ -- we'll let `SymPy` do the heavy lifting :)

In [2]:
w = sym.symbols(r'\omega')

A = sym.Matrix([[0,1,0,0],[3*w**2,0,0,2*w],[0,0,0,1],[0,-2*w,0,1]])
B = sym.Matrix([[0,0],[1,0],[0,0],[0,1]])

A @ B

⎡    1         0    ⎤
⎢                   ⎥
⎢    0      2⋅\omega⎥
⎢                   ⎥
⎢    0         1    ⎥
⎢                   ⎥
⎣-2⋅\omega     1    ⎦

The columns of $AB$ are clearly linearly independent, and clearly linearly independent from those of $B$ (look at the patterns of zeros and ones -- each column has a 1 in a row where all other columns are zero).  We conclude that $\mathcal{C}$ is full rank, so the system is controllable.

(b.) If the radial thruster fails, is the system still controllable?

***Solution:*** the most natural interpretation of how the model changes if the radial thruster fails is that the first column of $B$ is set to zero; since a column of zeros is linearly ***de***pendent on every vector, we can simply discard this column when constructing the new controllability matrix -- again we'll let `SymPy` do the work for us:

In [24]:
b = B[:,1]
C = sym.Matrix.hstack(b,A @ b, A @ A @ b, A @ A @ A @ b)
C

⎡0     0         2⋅\omega             2⋅\omega       ⎤
⎢                                                    ⎥
⎢                                        3           ⎥
⎢0  2⋅\omega     2⋅\omega      - 2⋅\omega  + 2⋅\omega⎥
⎢                                                    ⎥
⎢                                           2        ⎥
⎢0     1             1            - 4⋅\omega  + 1    ⎥
⎢                                                    ⎥
⎢                       2                   2        ⎥
⎣1     1      - 4⋅\omega  + 1     - 8⋅\omega  + 1    ⎦

The first two columns are clearly linearly independent, and the third is clearly linearly independent from the first two so long as $\omega \neq 0$ -- if $\omega = 0$ the last two columns are linearly dependent on the first two:

In [25]:
v = sym.Matrix.nullspace(C.subs({w:0}))

v

⎡⎡0 ⎤  ⎡0 ⎤⎤
⎢⎢  ⎥  ⎢  ⎥⎥
⎢⎢-1⎥  ⎢-1⎥⎥
⎢⎢  ⎥, ⎢  ⎥⎥
⎢⎢1 ⎥  ⎢0 ⎥⎥
⎢⎢  ⎥  ⎢  ⎥⎥
⎣⎣0 ⎦  ⎣1 ⎦⎦

It turns out the last column is linearly independent from the first two so long as $\omega \neq 0$:

In [26]:
sym.Matrix.nullspace(C)

[]

We conclude that the system is controllable if the radial thruster fails (leaving only the tangential thruster) so long as the angular velocity $\omega$ is nonzero.

(c.) If the tangential thruster fails, is the system still controllable?



***Solution:*** the most natural interpretation of how the model changes if the tangential thruster fails is that the second column of $B$ is set to zero; since a column of zeros is linearly ***de***pendent on every vector, we can simply discard this column when constructing the new controllability matrix -- again we'll let `SymPy` do the work for us:

In [27]:
b = B[:,0]
C = sym.Matrix.hstack(b,A @ b, A @ A @ b, A @ A @ A @ b)
C

⎡                                                2                ⎤
⎢0      1          0                      -\omega                 ⎥
⎢                                                                 ⎥
⎢                     2                           2               ⎥
⎢1      0      -\omega                   -4⋅\omega                ⎥
⎢                                                                 ⎥
⎢0      0      -2⋅\omega                 -2⋅\omega                ⎥
⎢                                                                 ⎥
⎢                                   3            ⎛          2    ⎞⎥
⎣0  -2⋅\omega  -2⋅\omega  - 6⋅\omega  - 2⋅\omega⋅⎝- 4⋅\omega  + 1⎠⎦

In [28]:
v = sym.Matrix.nullspace(C)[0]

v

⎡        2⎤
⎢3⋅\omega ⎥
⎢         ⎥
⎢       2 ⎥
⎢ \omega  ⎥
⎢         ⎥
⎢   -1    ⎥
⎢         ⎥
⎣    1    ⎦

In [29]:
sym.simplify(C @ v)

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

We conclude that the system is ***not*** controllable if the radial thruster fails regardless of the angular velocity $\omega$.

# 2. controllable decomposition

Consider the LTI-DE 

$$ \dot{x}/x^+ = A x + B u,\ x\in\mathbb{R}^n,\ u\in\mathbb{R}^k, $$

and the LTI-DE obtained by applying the change-of-coordinates $\bar{x} = T^{-1} x$,

$$ \dot{\bar{x}}/\bar{x}^+ = \bar{A} \bar{x} + \bar{B} u,\ \bar{A} = T^{-1} A T,\ \bar{B} = T^{-1} B. $$

(a.) *Show that the rank of the controllability matrix $\mathcal{C}$ obtained from $(A,B)$ equals the rank of the controllability matrix $\bar{\mathcal{C}}$ obtained from $(\bar{A},\bar{B})$ (this implies, in particular, that $(A,B)$ is controllable if and only if $(\bar{A},\bar{B})$ is controllable).*

***Solution:*** it is straightforward to verify that 

$$ \mathcal{C} = T^{-1} \bar{\mathcal{C}}; $$

since $T$ is invertible, we conclude that $\operatorname{rank}\mathcal{C} = \operatorname{rank}\bar{\mathcal{C}}$.


Let $\mathcal{R}(\mathcal{C})\subset\mathbb{R}^n$ denote the "controllable subspace" for (LTI-DE), and let $\eta = \dim\mathcal{R}(\mathcal{C})$ denote the dimension of this subspace.

(b.) *Show that $\mathcal{R}(\mathcal{C})$ is "$A$-invariant" (***Hint:*** use the [Cayley-Hamilton Theorem](https://en.wikipedia.org/wiki/Cayley%E2%80%93Hamilton_theorem)):*

$$ \forall v\in\mathcal{R}(\mathcal{C}) : A v\in\mathcal{R}(\mathcal{C}). $$

***Solution:*** if $v\in\mathcal{R}(\mathcal{C})$ then we can write $v$ as a linear combination of columns of the collection of matrices $\{ A^\ell B \}_{\ell=0}^{n-1}$, that is $\exists w$ such that $v = \mathcal{C} w$, so we can write

$$ v = \mathcal{C} w = \sum_{\ell=0}^{n-1} A^\ell B w_\ell $$

where $w_\ell\in\mathbb{R}^n$ is the $\ell$-th block of $w\in\mathbb{R}^{nk}$.  This implies

$$ A v = \mathcal{C} w = \sum_{\ell=0}^{n-1} A^{\ell+1} B w_\ell. $$

By the [Cayley-Hamilton Theorem](https://en.wikipedia.org/wiki/Cayley%E2%80%93Hamilton_theorem), there exist coefficients $\{ a_\ell \}_{\ell=0}^{n-1}$ such that $A^n = \sum_{\ell=0}^{n-1} a_\ell A^\ell$.

This ensures we can write $Av$ as a linear combination of the columns of the collection of matrices $\{ A^\ell B \}_{\ell=0}^{n-1}$, i.e. $Av \in\mathcal{R}(\mathcal{C})$.

(c.) *Show that $\mathcal{R}(\mathcal{C})$ contains $\mathcal{R}(B)$.*

***Solution:*** if $v\in\mathcal{R}(B)$ then $\exists w_0\in\mathbb{R}^k$ such that $v = B w_0$; since $v = \mathcal{C} w$ where $w\in\mathbb{R}^{nk}$ is the vector obtained by vertically concatenating $w_0$ and $0\in\mathbb{R}^{(n-1)k}$, we conclude $v\in\mathcal{R}(\mathcal{C})$.

Let $V\in\mathbb{R}^{n\times\eta}$ be such that $\mathcal{R}(V) = \mathcal{R}(\mathcal{C})$, that is, the columns of $V$ form a basis for the controllable subspace.

(d.) *Use the results of (b.) and (c.) to show that there exist $A_c$, $B_c$ such that*

$$ A V = V A_c,\ B = V B_c. $$

(d., continued) *Be sure to specify the dimensions of $A_c$, $B_c$.*

***Solution:*** for the matrix multiplication to work out, we must have $A_c\in\mathbb{R}^{\eta\times\eta}$, $B_c\in\mathbb{R}^{\eta\times k}$.  Since $\mathcal{R}(\mathcal{C})$ is "$A$-invariant", the columns of $A V$ all belong to $\mathcal{R}(\mathcal{C})$, which means each column of $A V$ can be written as a linear combination of the columns of $V$ -- $A_c$ is the matrix whose columns specify these linear combinations; a similar argument applies to show the existence of $B_c$.



Let $U\in\mathbb{R}^{n\times(n-\eta)}$ be such that

$$ T = \left[\begin{array}{cc} V & U \end{array}\right] $$

is nonsingular, that is, the columns of $V$ and $U$ together form a basis for $\mathbb{R}^n$.

Finally, let

$$ T^{-1} A U = \left[\begin{array}{c} A_{cu} \\ A_u \end{array} \right] $$

where $A_{cu}\in\mathbb{R}^{\eta\times(n-\eta)}$, $A_u\in\mathbb{R}^{(n-\eta)\times(n-\eta)}$.

(e.) *Determine $\bar{A} = T^{-1} A T$ and $\bar{B} = T^{-1} B$ in terms of $A_c$, $A_{cu}$, $A_u$, and $B_c$.*

***Solution:*** it is straightforward to verify that

$$ \bar{A} = T^{-1} A T = \left[\begin{array}{cc} A_c & A_{cu} \\ 0 & A_u \end{array} \right],\ \bar{B} = T^{-1} B = \left[\begin{array}{c} B_c \\ 0 \end{array} \right]. $$


It is clear in (e.) that $\bar{x}_j$ for $j\in\{1,\dots,\eta\}$ are controllable and $\bar{x}_k$ for $k\in\{\eta+1,\dots,n\}$ are uncontrollable -- the "uncontrolled" states aren't affected by either the input or the "controlled" states; the pair $(A_c,B_c)$ is controllable by construction.

This change-of-coordinates is called the ***controllable decomposition.***


# 3. eigenvalue assignment

Consider the single-input/single-output (SISO) LTI-DE in ***controllable-canonical form***

$$ \dot{x}/x^+ = A x + B u,\ x\in\mathbb{R}^n,\ u\in\mathbb{R}^1, $$

$$ A = \left[\begin{array}{ccccc} -a_1 & -a_2 & \cdots & -a_{n-1} & -a_n \\ 1 & 0 & \cdots & 0 & 0 \\ 0 & 1 & \cdots & 0 & 0 \\ \vdots & \vdots & \ddots & \vdots & \vdots \\ 0 & 0 & \cdots & 1 & 0 \end{array}\right],\ B = \left[\begin{array}{c} 1 \\ 0 \\ \vdots \\ 0 \\ 0 \end{array}\right].
 $$

(a.) Show that the controllability matrix $\mathcal{C}$ obtained from $(A,B)$ has full rank, so the LTI-DE is controllable.

***Solution:*** it is straightforward to verify that 

$$ \mathcal{C} = \left[\begin{array}{ccccc} B & A B & A^2 B & \cdots & A^{n-1} B \end{array} \right] = \left[\begin{array}{ccccc} 1 & \ast & \ast & \cdots & \ast \\ 0 & 1 & \ast & \cdots & \ast \\ 0 & 0 & 1 & \cdots & \ast \\ \vdots & \vdots & \vdots & \ddots & \vdots \\ 0 & 0 & 0 & \cdots & 1 \end{array} \right], $$

where "$\ast$" is an unspecified scalar, so $\operatorname{rank}\mathcal{C} = n$.


(b.) Determine the characteristic polynomial of the closed-loop system obtained from linear state feedback $u = - K x$, $K = \left[\begin{array}{cccc} k_1 & k_2 & \cdots & k_n \end{array}\right]$.

***Solution:*** the closed-loop system matrix obtained from $u = - K$ is 

$$ A - B K = \left[\begin{array}{ccccc} -k_1-a_1 & -k_2-a_2 & \cdots & -k_{n-1}-a_{n-1} & -k_n-a_n \\ 1 & 0 & \cdots & 0 & 0 \\ 0 & 1 & \cdots & 0 & 0 \\ \vdots & \vdots & \ddots & \vdots & \vdots \\ 0 & 0 & \cdots & 1 & 0 \end{array}\right], $$

whose characteristic polynomial 

$$ \det(s I - (A - BK)) = s^n + (a_1+k_1) s^{n-1} + (a_2+k_2) s^{n-2} + \cdots + (a_{n-1}+k_{n-1}) s + (a_n+k_n) $$

can be obtained by recursively performing [Laplace expansions](https://en.wikipedia.org/wiki/Laplace_expansion) along the last column of $s I - (A - B K)$ (the determinant of each matrix minor is easy to compute).

(c.) Determine a polynomial that has roots $\lambda_1,\lambda_2,\dots,\lambda_n$.

***Solution:*** $(s - \lambda_1)(s - \lambda_2) \cdots (s - \lambda_n)$ has roots $\lambda_1,\lambda_2,\dots,\lambda_n$.

(d.) Specify an algorithm (i.e. a finite sequence of calculations each of which can be implemented on a digital computer) to determine $K$ to place the eigenvalues of the closed-loop system at $\lambda_1,\lambda_2,\dots,\lambda_n$.

***Solution:*** since the roots of a polynomial are uniquely determined by the sequence of coefficients that multiply powers of the variable $s$, and since multiplying monomials is easier than factoring polynomials, my proposed algorithm expands the desired characteristic polynomial from (c.) to obtain

$$ (s - \lambda_1)(s - \lambda_2) \cdots (s - \lambda_3) = s^n + \gamma_1 s^{n-1} + \cdots + \gamma_{n-1} s + \gamma_n $$

and then sets $k_\ell = \gamma_\ell - a_\ell$ for each $\ell\in\{1,\dots,n\}$.  Note that the coefficients $\{ \gamma_\ell \}_{\ell=1}^n$ can be determined in $O(n^2)$ steps.